<a href="https://colab.research.google.com/github/205J/CSE278/blob/main/MyHW3Solution.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [4]:
import numpy as np
from urllib import request
import gzip
import pickle

filename = [
["training_images","train-images-idx3-ubyte.gz"],
["test_images","t10k-images-idx3-ubyte.gz"],
["training_labels","train-labels-idx1-ubyte.gz"],
["test_labels","t10k-labels-idx1-ubyte.gz"]
]

def download_mnist():
    base_url = "http://yann.lecun.com/exdb/mnist/"
    for name in filename:
        print("Downloading "+name[1]+"...")
        request.urlretrieve(base_url+name[1], name[1])
    print("Download complete.")

def save_mnist():
    mnist = {}
    for name in filename[:2]:
        with gzip.open(name[1], 'rb') as f:
            mnist[name[0]] = np.frombuffer(f.read(), np.uint8, offset=16).reshape(-1,28*28)
    for name in filename[-2:]:
        with gzip.open(name[1], 'rb') as f:
            mnist[name[0]] = np.frombuffer(f.read(), np.uint8, offset=8)
    with open("mnist.pkl", 'wb') as f:
        pickle.dump(mnist,f)
    print("Save complete.")

def init():
    download_mnist()
    save_mnist()
#    print ((load()[0]).shape)
def load():
    with open("mnist.pkl",'rb') as f:
        mnist = pickle.load(f)
    return mnist["training_images"], mnist["training_labels"], mnist["test_images"], mnist["test_labels"]

if __name__ == '__main__':
    init()

Download complete.
Save complete.


In [5]:
import numpy as np
from urllib import request
import gzip
import math
import pickle


def grad_softmax_crossentropy(X, y):
    m = y.shape[0]
    ones_for_answers = np.zeros_like(X)
    ones_for_answers[np.arange(len(X)), y] = 1

    p = np.exp(X) / np.exp(X).sum(axis=-1, keepdims=True)
    return (- ones_for_answers + p) / m

def load():
    with open("mnist.pkl",'rb') as f:
        mnist = pickle.load(f)

        training_images, training_labels, testing_images, testing_labels = mnist["training_images"], mnist["training_labels"], mnist["test_images"], mnist["test_labels"]
        # Normalize the images
        training_images.astype('float32')
        testing_images.astype('float32')
        training_images = training_images / 255
        testing_images = testing_images / 255
        return training_images, training_labels, testing_images, testing_labels


def softmax(z):
    return np.exp(z) / np.sum(np.exp(z))


def softmax_prime(z):
    return softmax(z) * (1 - softmax(z))


def relu(z):
    return np.maximum(z, 0)


# return 1 or 0
def relu_prime(z):
    return (z > 0)


#     Forward equations:
#     z1 = x.w1+b1
#     a1 = relu(z1)

#     z2 = a1.w2+b2
#     a2 = relu(z2)

#     z3 = a2.w3+b3
#     a3 = softmax(z3)

#     Back propagation equations:
#     a3_delta = a3-y

#     z2_delta = a3_delta.w3.T
#     a2_delta = z2_delta.relu_prime(a2)

#     z1_delta = a2_delta.w2.T
#     a1_delta = z1_delta.relu_prime(a1)



def softmax_crossentropy(X, y):
    m = y.shape[0]
    p = softmax(X)
    log_likelihood = -np.log(p[range(m), y])
    loss = np.sum(log_likelihood) / m
    return loss





from time import time




TRimg,TRlab,TSimg,TSlab=load()
print(len(TRimg),len(TRlab),len(TSimg),len(TSlab))
print(len(TRimg[0]),len(TRlab),len(TSimg[0]),len(TSlab))

#hyper parameters
TotalTrainingData=len(TRimg)
BatchSize=128   #batchsize
epochs=10   #epochs times
IterationInOneEpoches=math.ceil(TotalTrainingData/BatchSize)
print(IterationInOneEpoches,BatchSize)

#network parameters
HidSize=[200,50]
ClassNO=10

#input layer
INPLayer=TRimg[0].copy()
print(len(INPLayer))
# W, b between inputlayer and hidden layer 1
WA=0.01*np.random.randn(len(INPLayer),HidSize[0])* np.sqrt(2 / (len(INPLayer) + HidSize[0]))
BA=np.zeros((1,HidSize[0]))+ 0.01
WB=0.01*np.random.randn(HidSize[0],HidSize[1])* np.sqrt(2 / (HidSize[0]+HidSize[1]))
BB=np.zeros((1,HidSize[1]))+ 0.01
WC=0.01*np.random.randn(HidSize[1],ClassNO)* np.sqrt(2 / (HidSize[1]+ClassNO))
BC=np.zeros((1,ClassNO))+ 0.01

LearningRate=0.1
reg=1e-3
BatchSize=128
INPLayer_B_Label=TRlab[0:BatchSize].copy()
INPLayer_B=TRimg[0:BatchSize].copy()
BatchSize=len(INPLayer_B)



# Load the dataset
file_name = 'mnist.pkl'

X_train, y_train, X_test, y_test = load()

batchsize = 32
start_time=time()
BatchSize=batchsize
for i in range(0,epochs):

    #input image & label
    for i in range(0, len(X_train) - batchsize + 1, batchsize):
        interval = slice(i, i + batchsize)

        x_batch, y_batch = X_train[interval], y_train[interval]
        #train(network, x_batch, y_batch)

        #input image & label for one batch
        INPLayer_B_Label=y_batch.copy()
        INPLayer_B=x_batch.copy()
        #FW:
        #print(len(INPLayer_B))
        #input -> hidden 1 relu
        HA=np.maximum(0,np.dot(INPLayer_B,WA)+BA)
        #print(HA.shape)
        # hidden 1 -> hidden 2 relu
        HB=np.maximum(0,np.dot(HA,WB)+BB)
        #print(HB.shape)
        # hidden 2 -> output
        OutPut=np.dot(HB,WC)+BC
        #print(OutPut.shape)
        #softmax

        Result=np.exp(OutPut)
        probs=Result/np.sum(Result,axis=1,keepdims=True)
        #print(probs)
        #print(np.sum(probs))
        #print(TRlab.shape)
        #compute the loss
        #print(len(INPLayer_B_Label))
        #print(INPLayer_B_Label)

        #predicted_class = np.argmax(OutPut, axis=1)
        #print('Training accuracy: ', np.mean(predicted_class == INPLayer_B_Label))

        #calculate loss
        Corect_logprobs=-np.log(probs[range(BatchSize),INPLayer_B_Label])
        Data_loss=np.sum(Corect_logprobs)/BatchSize
        reg_loss=0.5*reg*np.sum(WA*WA)+0.5*reg*np.sum(WB*WB)+0.5*reg*np.sum(WC*WC)
        LOSS=Data_loss+reg_loss

        NDATALOSS=softmax_crossentropy(OutPut,INPLayer_B_Label)
        #print("loss in mine:",NDATALOSS)

        #BW:
        '''
        Dscores=probs.copy()
        Dscores[range(BatchSize),INPLayer_B_Label]-=1
        Dscores/=BatchSize
        '''

        Dscores=grad_softmax_crossentropy(OutPut,INPLayer_B_Label)
        #print("degra in my: ",Dscores)
        DWC=np.transpose(np.dot(np.transpose(Dscores),HB))
        #print("degra in my: ", DWC)
        DBC=np.sum(Dscores,axis=0)          #,keepdims=True
        #print("degra in my: ", DBC)
        DhiddenB=np.dot(Dscores,WC.T)
        #print("degra in my: ", DhiddenB)

        DhiddenB[HB <= 0] = 0
        #print("degra in my: ", DhiddenB)

        WC += -LearningRate * DWC
        #print("degra in my: ", WC)

        BC -= LearningRate * DBC
        #print("degra in my: ", BC)


        #DWB=np.dot(HA.T,DhiddenB)
        DWB=np.transpose(np.dot(np.transpose(DhiddenB),HA))

        DBB=np.sum(DhiddenB,axis=0)



        #DhiddenA=np.dot(HB,WB.T)        #np.dot(Dscores,WC.T)  DhiddenB
        #print("my input data is : ", WB, "====", DhiddenB)
        DhiddenA = np.dot(DhiddenB, np.transpose(WB))

        DhiddenA[HA <=0]=0

        WB += -LearningRate * DWB
        BB += -LearningRate * DBB


        #DWA=np.dot(INPLayer_B.T,DhiddenA)
        #np.transpose(np.dot(np.transpose(DhiddenB),HA))
        DWA =np.transpose(np.dot(np.transpose(DhiddenA),INPLayer_B))
        DBA=np.sum(DhiddenA,axis=0)
        WA += -LearningRate * DWA
        BA += -LearningRate * DBA
        #print("degra in my: ", BB)
        #exit()

        '''
        DWA += reg*WA
        DWB += reg * WB
        DWC += reg * WC

        WA += -LearningRate*DWA
        BA += -LearningRate*DBA

        WC += -LearningRate*DWC
        BC += -LearningRate*DBC
        '''

    print(i)
print("Training Time:", time()-start_time, "sec")
#Training accuracy
INPLayer_C=TRimg.copy()
INPLayer_C_Label=TRlab.copy()
HA=np.maximum(0,np.dot(INPLayer_C,WA)+BA)
#print(HA.shape)
HB=np.maximum(0,np.dot(HA,WB)+BB)
#print(HB.shape)
OutPut=np.dot(HB,WC)+BC
#print(OutPut.shape)
Result=np.exp(OutPut)
probs=Result/np.sum(Result,axis=1,keepdims=True)


predicted_class = np.argmax(OutPut, axis=1)
print( 'Training accuracy: ',np.mean(predicted_class == INPLayer_C_Label))


# evaluate training set accuracy
INPLayer_C=TSimg.copy()
INPLayer_C_Label=TSlab.copy()
HA=np.maximum(0,np.dot(INPLayer_C,WA)+BA)
#print(HA.shape)
HB=np.maximum(0,np.dot(HA,WB)+BB)
#print(HB.shape)
OutPut=np.dot(HB,WC)+BC
#print(OutPut.shape)
Result=np.exp(OutPut)
probs=Result/np.sum(Result,axis=1,keepdims=True)


predicted_class = np.argmax(OutPut, axis=1)
print( 'Testing accuracy: ',np.mean(predicted_class == INPLayer_C_Label))

#for hidden layer

60000 60000 10000 10000
784 60000 784 10000
469 128
784
59968
59968
59968
59968
59968
59968
59968
59968
59968
59968
Training Time: 39.52688932418823 sec
Training accuracy:  0.9891
Testing accuracy:  0.9747


In [ ]:
from __future__ import print_function

import numpy as np
from urllib import request
import gzip
import math
import pickle
from time import time

import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import datasets, transforms
from torch.autograd import Variable



# Training settings batch_size & learning rate
batch_size = 32
lr=0.01


# MNIST Dataset
train_dataset = datasets.MNIST(root='./mnist_data/',
                               train=True,
                               transform=transforms.ToTensor(),
                               download=True)

test_dataset = datasets.MNIST(root='./mnist_data/',
                              train=False,
                              transform=transforms.ToTensor())

# Data Loader (Input Pipeline)
train_loader = torch.utils.data.DataLoader(dataset=train_dataset,
                                           batch_size=batch_size,
                                           shuffle=True)

test_loader = torch.utils.data.DataLoader(dataset=test_dataset,
                                          batch_size=batch_size,
                                          shuffle=False)

class Net(nn.Module):

    def __init__(self):
        super(Net, self).__init__()
        self.l1 = nn.Linear(784, 200)
        self.l2 = nn.Linear(200, 50)
        self.l3 = nn.Linear(50, 10)


    def forward(self, x):
        x = x.view(-1, 784)  # Flatten the data (n, 1, 28, 28)-> (n, 784)
        x = F.relu(self.l1(x))
        x = F.relu(self.l2(x))

        return F.log_softmax(x, dim=1)

#read data
def load():
    with open("mnist.pkl",'rb') as f:
        mnist = pickle.load(f)

        training_images, training_labels, testing_images, testing_labels = mnist["training_images"], mnist["training_labels"], mnist["test_images"], mnist["test_labels"]
        # Normalize the images
        training_images.astype('float32')
        testing_images.astype('float32')
        training_images = training_images / 255
        testing_images = testing_images / 255
        return training_images, training_labels, testing_images, testing_labels


#get the data from the data set
TRimg,TRlab,TSimg,TSlab=load()
print(len(TRimg),len(TRlab),len(TSimg),len(TSlab))
print(len(TRimg[0]),len(TRlab),len(TSimg[0]),len(TSlab))

#using pytorch to build the network
model = Net()

criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(model.parameters(), lr=0.01, momentum=0.5)


#

def Train(epoch):
    model.train()
    for batch_idx, (data, target) in enumerate(train_loader):
        data, target = Variable(data), Variable(target)
        optimizer.zero_grad()
        output = model(data)
        loss = criterion(output, target)
        loss.backward()
        optimizer.step()
        if batch_idx % 10 == 0:


            print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(
            epoch, batch_idx * len(data), len(train_loader.dataset),
            100. * batch_idx / len(train_loader), loss.data))



def Test():
    model.eval()
    test_loss = 0
    correct = 0
    for data, target in test_loader:
        data, target = Variable(data, volatile=True), Variable(target)
        output = model(data)
        # sum up batch loss
        #test_loss += criterion(output, target).data[0]
        test_loss += criterion(output, target).data
        # get the index of the max
        pred = output.data.max(1, keepdim=True)[1]
        correct += pred.eq(target.data.view_as(pred)).cpu().sum()

    test_loss /= len(test_loader.dataset)
    print('\nTest set: Average loss: {:.4f}, Accuracy: {}/{} ({:.0f}%)\n'.format(
        test_loss, correct, len(test_loader.dataset),
        100. * correct / len(test_loader.dataset)))
start_time = time()
for epoch in range(1, 10):

    Train(epoch)

    Test()
print("Training Time is:", time()-start_time)

  0%|          | 0/9912422 [00:00<?, ?it/s]

Extracting ./mnist_data/MNIST/raw/train-images-idx3-ubyte.gz to ./mnist_data/MNIST/raw



  0%|          | 0/28881 [00:00<?, ?it/s]

Extracting ./mnist_data/MNIST/raw/train-labels-idx1-ubyte.gz to ./mnist_data/MNIST/raw



  0%|          | 0/1648877 [00:00<?, ?it/s]

Extracting ./mnist_data/MNIST/raw/t10k-images-idx3-ubyte.gz to ./mnist_data/MNIST/raw



  0%|          | 0/4542 [00:00<?, ?it/s]

Extracting ./mnist_data/MNIST/raw/t10k-labels-idx1-ubyte.gz to ./mnist_data/MNIST/raw

60000 60000 10000 10000
784 60000 784 10000
Train Epoch: 1 [0/60000 (0%)]	Loss: 3.898158
Train Epoch: 1 [320/60000 (1%)]	Loss: 3.827191
Train Epoch: 1 [640/60000 (1%)]	Loss: 3.726692
Train Epoch: 1 [960/60000 (2%)]	Loss: 3.521844
Train Epoch: 1 [1280/60000 (2%)]	Loss: 3.334667
Train Epoch: 1 [1600/60000 (3%)]	Loss: 3.034194
Train Epoch: 1 [1920/60000 (3%)]	Loss: 2.857593
Train Epoch: 1 [2240/60000 (4%)]	Loss: 2.489261
Train Epoch: 1 [2560/60000 (4%)]	Loss: 2.173745
Train Epoch: 1 [2880/60000 (5%)]	Loss: 2.129928
Train Epoch: 1 [3200/60000 (5%)]	Loss: 1.988209
Train Epoch: 1 [3520/60000 (6%)]	Loss: 1.864345
Train Epoch: 1 [3840/60000 (6%)]	Loss: 1.843202
Train Epoch: 1 [4160/60000 (7%)]	Loss: 1.627957
Train Epoch: 1 [4480/60000 (7%)]	Loss: 1.422466
Train Epoch: 1 [4800/60000 (8%)]	Loss: 1.515055
Train Epoch: 1 [5120/60000 (9%)]	Loss: 1.357608
Train Epoch: 1 [5440/60000 (9%)]	Loss: 1.298200
Train Epoch

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:110: UserWarning: volatile was removed and now has no effect. Use `with torch.no_grad():` instead.



Test set: Average loss: 0.0102, Accuracy: 9089/10000 (91%)

Train Epoch: 2 [0/60000 (0%)]	Loss: 0.424406
Train Epoch: 2 [320/60000 (1%)]	Loss: 0.116542
Train Epoch: 2 [640/60000 (1%)]	Loss: 0.098097
Train Epoch: 2 [960/60000 (2%)]	Loss: 0.345943
Train Epoch: 2 [1280/60000 (2%)]	Loss: 0.188013
Train Epoch: 2 [1600/60000 (3%)]	Loss: 0.586893
Train Epoch: 2 [1920/60000 (3%)]	Loss: 0.291219
Train Epoch: 2 [2240/60000 (4%)]	Loss: 0.345831
Train Epoch: 2 [2560/60000 (4%)]	Loss: 0.755577
Train Epoch: 2 [2880/60000 (5%)]	Loss: 0.166588
Train Epoch: 2 [3200/60000 (5%)]	Loss: 0.349934
Train Epoch: 2 [3520/60000 (6%)]	Loss: 0.247151
Train Epoch: 2 [3840/60000 (6%)]	Loss: 0.435447
Train Epoch: 2 [4160/60000 (7%)]	Loss: 0.546932
Train Epoch: 2 [4480/60000 (7%)]	Loss: 0.429912
Train Epoch: 2 [4800/60000 (8%)]	Loss: 0.279270
Train Epoch: 2 [5120/60000 (9%)]	Loss: 0.373389
Train Epoch: 2 [5440/60000 (9%)]	Loss: 0.316829
Train Epoch: 2 [5760/60000 (10%)]	Loss: 0.243765
Train Epoch: 2 [6080/60000 (10%)